In [ ]:
from shipping import *

In this example we are using the volume_ft3 property in the RefrigereatedShippingContainer subclass to override the corresponding property in the ShippingContainer base class.

In [ ]:
class RefrigeratedShippingContainer(ShippingContainer):

    MAX_CELSIUS = 4.0
    FRIDGE_VOLUME_FT3 = 100
    @staticmethod
    def _make_bic_code(owner_code, serial):
        return iso6346.create(owner_code=owner_code, serial=str(serial).zfill(6), category='R')

    @staticmethod
    def _c_to_f(celsius):
        return celsius * 9/5 + 32

    @staticmethod
    def _f_to_c(fahrenheit):
        return (fahrenheit - 32) * 5/9

    def __init__(self, owner_code, length_ft, contents, celsius):
        super().__init__(owner_code, length_ft, contents)
        self.celsius = celsius

    @property
    def celsius(self):
        return self._celsius 

    @celsius.setter
    def celsius(self, value):
        if value > RefrigeratedShippingContainer.MAX_CELSIUS:
            raise ValueError("Temperature too hot!")
        self._celsius = value
    
    @property
    def fahrenheit(self):
        return RefrigeratedShippingContainer._c_to_f(self.celsius)

    @fahrenheit.setter
    def fahrenheit(self, value):
        self.celsius = RefrigeratedShippingContainer._f_to_c(value)

    @property
    def volume_ft3(self):
        return (self.length_ft * ShippingContainer.HEIGHT_FT * ShippingContainer.WIDTH_FT - RefrigeratedShippingContainer.FRIDGE_VOLUME_FT3)

In [ ]:
r = RefrigeratedShippingContainer.create_empty('YML', length_ft=20, celsius=-10.0)

In [ ]:
c = ShippingContainer.create_empty('YML', length_ft=20)

In [ ]:
r.volume_ft3

In [ ]:
c.volume_ft3

This now works for the RefrigeratedShippingContainer subclass, however there is now a duplication of the bulk volume calculation between the overridden property and its base class implementation.  This can be addressed by having the derived class version delegate to the base class.  This is done by using super() to retrieve the base-class property:

In [ ]:
class RefrigeratedShippingContainer(ShippingContainer):

    MAX_CELSIUS = 4.0
    FRIDGE_VOLUME_FT3 = 100
    @staticmethod
    def _make_bic_code(owner_code, serial):
        return iso6346.create(owner_code=owner_code, serial=str(serial).zfill(6), category='R')

    @staticmethod
    def _c_to_f(celsius):
        return celsius * 9/5 + 32

    @staticmethod
    def _f_to_c(fahrenheit):
        return (fahrenheit - 32) * 5/9

    def __init__(self, owner_code, length_ft, contents, celsius):
        super().__init__(owner_code, length_ft, contents)
        self.celsius = celsius

    @property
    def celsius(self):
        return self._celsius 

    @celsius.setter
    def celsius(self, value):
        if value > RefrigeratedShippingContainer.MAX_CELSIUS:
            raise ValueError("Temperature too hot!")
        self._celsius = value
    
    @property
    def fahrenheit(self):
        return RefrigeratedShippingContainer._c_to_f(self.celsius)

    @fahrenheit.setter
    def fahrenheit(self, value):
        self.celsius = RefrigeratedShippingContainer._f_to_c(value)

    @property
    def volume_ft3(self):
        return (super().volume_ft3 - RefrigeratedShippingContainer.FRIDGE_VOLUME_FT3)

In [ ]:
r2 =  RefrigeratedShippingContainer.create_empty('YML', length_ft=20, celsius=-10.0)

In [ ]:
r2.volume_ft3

The property has been successfully redefined in the derived class, and delegation to the base class is possible if necessary